In [1]:
import numpy as np
import librosa as lr
import tensorflow as tf

In [2]:
def _stft_tensorflow(self, waveform):
    return self.power_to_db(
        tf.matmul(
            tf.square(
                tf.abs(
                    tf.signal.stft(
                        np.squeeze(waveform),
                        frame_length=2048,
                        frame_step=1024,
                        fft_length=2048
                    )
                )
            ),
            tf.signal.linear_to_mel_weight_matrix(
                num_mel_bins=128,
                num_spectrogram_bins=2048 // 2 + 1,
                sample_rate=44100,
                lower_edge_hertz=0,
                upper_edge_hertz=8000
            )
        )
    )


""" Source: https://gist.github.com/dschwertfeger/f9746bc62871c736e47d5ec3ff4230f7 """
def _power_to_db(self, S, amin=1e-16, top_db=80.0):
    """Convert a power-spectrogram (magnitude squared) to decibel (dB) units.
    Computes the scaling ``10 * log10(S / max(S))`` in a numerically
    stable way.
    Based on:
    https://librosa.github.io/librosa/generated/librosa.core.power_to_db.html
    """
    def _tf_log10(x):
        numerator = tf.math.log(x)
        denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
        return numerator / denominator
    
    # Scale magnitude relative to maximum value in S. Zeros in the output 
    # correspond to positions where S == ref.
    ref = tf.reduce_max(S)

    log_spec = 10.0 * _tf_log10(tf.maximum(amin, S))
    log_spec -= 10.0 * _tf_log10(tf.maximum(amin, ref))

    log_spec = tf.maximum(log_spec, tf.reduce_max(log_spec) - top_db)

    return log_spec


    
def _stft_librosa(self, waveform):
    return lr.stft(
        lr.to_mono(
            waveform.reshape( 
                (waveform.shape[1], waveform.shape[0]) 
            )
        ),
        sr=44100,
        n_fft=2048,
        hop_length=1024
    )